In [1]:
# from data_multi import Bandit_multi
# from learner_diag import NeuralUCBDiag
import numpy as np
import argparse
import pickle
import os
import time
import torch
import numpy as np
from scipy.stats import truncnorm
import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 


def truncated_gaussian(mean, variance, a, b, size):
    # Calculate the standard deviation from the variance
    std_dev = np.sqrt(variance)

    # Calculate the lower and upper bounds for truncation
    lower_bound = (a - mean) / std_dev
    upper_bound = (b - mean) / std_dev

    # Generate samples from the truncated normal distribution
    samples = truncnorm.rvs(lower_bound, upper_bound, loc=mean, scale=std_dev, size=size)

    return np.array(samples)

def reshape_context(context, A):
    d = context.shape[0]
    reshaped_context = np.zeros((A, d * A))
    
    for i in range(A):
        start_idx = i * d
        end_idx = start_idx + d
        reshaped_context[i, start_idx:end_idx] = context
    
    return reshaped_context

class Evaluation:

    def __init__(self, game_name, task, n_folds, horizon, game, label, context_type):
        self.game_name = game_name
        self.task = task
        self.n_folds = n_folds
        self.horizon = horizon
        self.game = game
        self.label =  label
        self.context_type = context_type
        

    def get_outcomes(self, game, ):
        outcomes = np.random.choice( game.n_outcomes , p= list( game.outcome_dist.values() ), size= self.horizon) 
        return outcomes

    def get_feedback(self, game, action, outcome):
        return game.FeedbackMatrix[ action ][ outcome ]

    def eval_policy_once(self, alg, game, job):

        

        context_generator, jobid = job

        np.random.seed(jobid)

        alg.reset(context_generator.d)

        cumRegret =  np.zeros(self.horizon, dtype =float)
        compteur = None
        for t in range(self.horizon):
            compteur = t
            if t % 1000 == 0 :
                print(t)

            context,distribution = context_generator.get_context()
            outcome = 0 if distribution[0]>0.5 else 1  #np.random.choice( 2 , p = distribution )

            context = reshape_context(context, alg.A) if alg.name == 'neuralcbpside' else np.reshape(context, (-1,1))
            
            
            action = alg.get_action(t, context)

            print('t', t, 'action', action, 'outcome', outcome, 'gaps', ( game.LossMatrix[0,...] - game.LossMatrix[1,...])  @ distribution  )

            feedback =  self.get_feedback( game, action, outcome )

            alg.update(action, feedback, outcome, t, context )

            i_star = np.argmin(  [ game.LossMatrix[i,...] @ np.array( distribution ) for i in range(alg.N) ]  )
            loss_diff = game.LossMatrix[action,...] - game.LossMatrix[i_star,...]
            val = loss_diff @ np.array( distribution )
            cumRegret[t] =  val

        return cumRegret
    
class ToyContexts:

    def __init__(self, ):
        self.type = 'toy'
        self.d_context = 2

    def get_context(self, ):
        p = np.random.uniform(0, 1)
        return np.array([1,p]), [p, 1-p]


In [4]:
import neuralcbpside_v1
import neuralcbpside_v2
import neuralcbpside_v3
import games
import synthetic_data
import numpy as np
import geometry_v3
import cbpside

horizon = 2500
lbd = 0.05
game = games.apple_tasting( )

size = 5
w = np.array([1/size]*size)
context_generator = synthetic_data.LinearContexts( w,'imbalanced' )

# w = np.array([0,1])
# context_generator = ToyContexts()

alg = neuralcbpside_v3.NeuralCBPside(game, 'simplified', 1.01, lbd, 10, 'cuda:0' )
eval2 = Evaluation(None, None, None, horizon, game, None, None)
job2 = context_generator, 0
cumRegret4 = eval2.eval_policy_once(alg, game, job2)

# alg = cbpside.CBPside( game,  1.01, lbd )
# eval1 = Evaluation(None, None, None, horizon, game, None, None)
# job1 = context_generator, 0
# cumRegret1 = eval1.eval_policy_once(alg, game, job1)

0
t 0 action 0 outcome 1 gaps -0.3980454121465575
t 1 action 1 outcome 0 gaps 0.17906267997405267
t 2 action 1 outcome 0 gaps 0.37825227978457665
t 3 action 1 outcome 0 gaps 0.5821782220204399
t 4 action 1 outcome 0 gaps 0.5348359102462159
t 5 action 1 outcome 0 gaps 0.2983022554631296
t 6 action 1 outcome 0 gaps 0.5208593415582639
t 7 action 1 outcome 0 gaps 0.3787712288920142
t 8 action 1 outcome 1 gaps -0.3960212669376481
t 9 action 1 outcome 1 gaps -0.4949791151808877
t 10 action 1 outcome 0 gaps 0.4942530657795059
t 11 action 1 outcome 0 gaps 0.4370045538159637
t 12 action 1 outcome 0 gaps 0.44854252631923797
t 13 action 1 outcome 0 gaps 0.14716485175744687
t 14 action 1 outcome 0 gaps 0.7331270889746433
t 15 action 1 outcome 0 gaps 0.35019746876933455
t 16 action 1 outcome 1 gaps -0.22522851833916485
t 17 action 1 outcome 1 gaps -0.6370012573962842
t 18 action 1 outcome 1 gaps -0.6390736744627654
t 19 action 1 outcome 1 gaps -0.47896683650628086
t 20 action 1 outcome 1 gaps -0.31

KeyboardInterrupt: 

In [12]:
import games
game = games.apple_tasting()
feedback = 2
np.where(game.FeedbackMatrix == feedback)[0][0]
np.unique(game.FeedbackMatrix)

array([0, 1, 2])

In [5]:
for alg_id, seed in enumerate(range(id, id+ncpus,1)):
    print(alg_id)

NameError: name 'ncpus' is not defined

In [37]:

import torch
import torch.nn as nn
import torch.optim as optim

# class Network(nn.Module):
#     def __init__(self, output_dim, dim, hidden_size=10):
#         super(Network, self).__init__()
#         self.fc1 = nn.Linear(dim, hidden_size)
#         self.activate = nn.ReLU()
#         self.fc2 = nn.Linear(hidden_size, output_dim)
#     def forward(self, x):
#         return self.fc2(self.activate(self.fc1(x)))

class Network(nn.Module):
    def __init__(self, output_dim, dim, hidden_size=10):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(dim, hidden_size)
        self.activate = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_dim)
        self.sigmoid = nn.Sigmoid( )
    def forward(self, x):
        x = self.fc2(self.activate(self.fc1(x)))
        x = self.sigmoid(x)
        return x

# Create an instance of your model
model = Network(1, 15).cuda()

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1) #, weight_decay=1

length = labels.shape[0]
index = np.arange(length)
np.random.shuffle(index)

epochs = 1000
for epoch in range(epochs):
    running_loss = 0.0
    for i in index:
        input, label = torch.tensor(features[i], dtype=torch.float).cuda() ,  torch.tensor(labels[i], dtype=torch.float).cuda() 
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        output = model(input)
        # print(output)
        # loss = nn.MSELoss()(output, label)
        loss = nn.BCELoss()(output, label)
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Track the loss
        running_loss += loss.item()

    # Print the average loss for the epoch
    print(f"Epoch {epoch + 1} Loss: {running_loss }")

print("Training finished.")

Epoch 1 Loss: 191.61819949746132
Epoch 2 Loss: 171.81261309236288
Epoch 3 Loss: 160.70064198412
Epoch 4 Loss: 151.76865445636213
Epoch 5 Loss: 144.26779567822814
Epoch 6 Loss: 139.35872966982424
Epoch 7 Loss: 135.98660619184375
Epoch 8 Loss: 133.81815046432894
Epoch 9 Loss: 133.40905308397487
Epoch 10 Loss: 129.53546123723208
Epoch 11 Loss: 128.15112248023797
Epoch 12 Loss: 127.52097333655547
Epoch 13 Loss: 127.6262948125077
Epoch 14 Loss: 124.9297045631356
Epoch 15 Loss: 124.34213168322094
Epoch 16 Loss: 125.65422660583954
Epoch 17 Loss: 125.57103053475021
Epoch 18 Loss: 125.53729559366275
Epoch 19 Loss: 124.15089444805108
Epoch 20 Loss: 123.72886606978
Epoch 21 Loss: 123.4920854429688
Epoch 22 Loss: 123.12331012448703
Epoch 23 Loss: 123.13061843031414
Epoch 24 Loss: 123.01522998188307
Epoch 25 Loss: 122.70486598303205
Epoch 26 Loss: 121.8172483319272
Epoch 27 Loss: 121.42813189881213
Epoch 28 Loss: 121.73620852425609
Epoch 29 Loss: 121.6918568751455
Epoch 30 Loss: 121.07351516577205


In [ ]:
# plt.plot(range(horizon), np.cumsum(cumRegret1) )
# plt.plot(range(horizon), np.cumsum(cumRegret2) )
plt.plot(range(horizon), np.cumsum(cumRegret3) )

In [31]:
features = None
labels = None

In [32]:
import numpy as np

class LinearContexts:
    def __init__(self,  w, task):
        self.d = len(w) # number of features
        self.w = w
        self.type = 'linear'
        self.task = task
    
    def get_context(self, ):

        if self.task == 'imbalanced':
            context = truncated_gaussian(0, 0.1, 0, 1, self.d) if np.random.uniform(0,1)<0.5 else truncated_gaussian(1, 0.1, 0, 1, self.d)
        else:
            context = truncated_gaussian(0.5, 1, 0, 1, self.d)
        # cont = context.reshape(self.d,1)
        p = self.w @ context
        val = [ p, 1-p ]
        context = context - np.ones(5) * 0.5
        context = context.astype(np.float64)
        return context , val 

A = 3
size = 5
w = np.array([1/size]*size)
context_generator = LinearContexts(w, 'imbalanced')

for _ in range(100):
    c,d = context_generator.get_context()
    print(d)
    c = reshape_context(c, A)

    e_y = np.zeros( A )
    outcome = 0 if d[0]<0.5 else 1
    action = np.random.randint(2)
    if action == 0:
        e_y[-1] = 1
    else:
        e_y[outcome] = 1
    print(e_y)
    Y_t = e_y.reshape(A,1)
    features = c if features is None else np.concatenate((features, c), axis=0)
    labels = Y_t if labels is None else np.concatenate((labels, Y_t), axis=0)


[0.781837598653284, 0.21816240134671605]
[0. 1. 0.]
[0.6556621380150109, 0.34433786198498906]
[0. 0. 1.]
[0.7789886507633803, 0.2210113492366197]
[0. 1. 0.]
[0.6950717806433822, 0.30492821935661785]
[0. 0. 1.]
[0.2905294738750467, 0.7094705261249533]
[0. 0. 1.]
[0.7637702218739297, 0.23622977812607027]
[0. 0. 1.]
[0.2930578692126846, 0.7069421307873154]
[0. 0. 1.]
[0.8009125498861533, 0.19908745011384665]
[0. 0. 1.]
[0.7022237121316804, 0.29777628786831956]
[0. 0. 1.]
[0.838445309974184, 0.16155469002581602]
[0. 1. 0.]
[0.8483303844579365, 0.15166961554206349]
[0. 1. 0.]
[0.37632054322920316, 0.6236794567707968]
[1. 0. 0.]
[0.81904645005698, 0.18095354994302004]
[0. 1. 0.]
[0.11547315940506402, 0.884526840594936]
[0. 0. 1.]
[0.20238851634306218, 0.7976114836569378]
[1. 0. 0.]
[0.16724887449258274, 0.8327511255074173]
[1. 0. 0.]
[0.276104891987722, 0.723895108012278]
[1. 0. 0.]
[0.6470652508863118, 0.35293474911368816]
[0. 1. 0.]
[0.7852475857655943, 0.21475241423440572]
[0. 0. 1.]
[0.7

In [33]:
d = 10
sigma_i = 2
lbd = 0.05
value1, value2, value3, value4 = [], [], [], []
for t in range(1, 10000):
    value1.append(  d * (  np.sqrt(  d * np.log(t) + 2 * np.log(1/t**2)   ) + 2 ) )
    value2.append(  d * (  np.sqrt(  (d+1) * np.log(t)   ) + 2 ) )
    value3.append(  sigma_i * (  np.sqrt(  d * np.log(t) + 2 * np.log(1/t**2)   ) +  sigma_i ) )
    value4.append(  sigma_i * (  np.sqrt(  d * np.log(t) + 2 * np.log(1/t**2)   ) + np.sqrt(lbd) * sigma_i ) )

plt.plot( range(9999), value1, label = 'width1')
plt.plot( range(9999), value2, label = 'width2')
plt.plot( range(9999), value3, label = 'width3')
plt.plot( range(9999), value4, label = 'width4')
plt.legend()

p = 435
sigma_i = 2
lbd = 0.05
value3, value4 = [], []
for t in range(1, 10000):
    value3.append(  sigma_i * (  np.sqrt(  p * np.log(t) + 2 * np.log(1/t**2)   ) +  sigma_i ) )
    value4.append(  sigma_i * (  np.sqrt(  p * np.log(t) + 2 * np.log(1/t**2)   ) + np.sqrt(lbd) * sigma_i ) )

plt.plot( range(9999), value3, label = 'width3')
plt.plot( range(9999), value4, label = 'width4')
plt.legend()

size = 10
w = np.array([1/size]*size)
context_generator = synthetic_data.LinearContexts( w,'balanced' )

dis = []
for _ in range(1000):
    c,d = context_generator.get_context()
    dis.append(d[0])

plt.hist(dis, bins=30, density=True, alpha=0.7)